# Sudoku Using a MIP Solver

NOTE: This notebook matches the SudokuMip notebook except that this uses `= 1` in the constraints.
This is a strong formulation which generally helps MIP solvers perform better.

Solve sudoku puzzles using a MIP (Mixed Integer linear Program) solver. We've integrated three solvers,
HiGHS, Gurobi, and GLPK. Gurobi requires a license to run so is only used in the final solve. If you don't
have a Gurobi license, that run will produce an error.

## Define a function to map a digit character to its value.

We define a function that maps from a Unicode character to its "value" from the
perspective of Sudoku. We want to support traditional Sudoku (of rank 3) as
well as higher ranks, up to rank 6, so we need a total of 36 symbols. Traditional
Sudoku uses `"123456789"`. To get 36 symbols, we'll also use `0` and `A`
through `Z`, so our symbols are `"1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ"`.

This function should map a Unicode character to its position in our symbol list,
and return `-1` if the Unicode character is not one of our symbols.

In [1]:
// Accepts a Unicode code point. Returns the index into
// "1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ", or -1 if not found.
// Note that 0 follows 9.

func ToDigit(ch) :=
  ch - "1"[0] if "1"[0] <= ch <= "9"[0]      else
  9           if           ch  = "0"[0]      else
  ch - "A"[0] + 10 if "A"[0] <= ch <= "Z"[0] else
  ch - "A"[0] + 10 if "A"[0] <= ch <= "Z"[0] else
  -1;

## Define the module.

The parameter `M` specifies the grid size value, with default of `3`. This can handle `M` values of `3`, `4`, `5`, and `6`.

In [2]:
Sudoku := module {
    param M := 3;
    const N := M * M;
    const NumCells := N * N;
    const NumMoves := NumCells * N;

    // All possible "moves" where "move" means "place a value in a cell".
    const Moves := Range(NumMoves)
        ->ForEach(as Id,
            With(cell: Id div N, row: cell div N, col: cell mod N,
                { Id, XRow: row, XCol: col, YBlk: row div M * M + col div M, ZVal: Id mod N }));

    // Group the moves in various ways. These are used for constraints below.
    // Each group of these represent, respectively:
    // * The possible values for a particular cell.
    // * The possible placements of a particular value in a particular row.
    // * The possible placements of a particular value in a particular column.
    // * The possible placements of a particular value in a particular block.
    const MovesByRowCol := Moves->GroupBy([key] _:XRow, [key] _:XCol);
    const MovesByValRow := Moves->GroupBy([key] _:ZVal, [key] _:XRow);
    const MovesByValCol := Moves->GroupBy([key] _:ZVal, [key] _:XCol);
    const MovesByValBlk := Moves->GroupBy([key] _:ZVal, [key] _:YBlk);

    // The fixed moves defined by a particular puzzle.
    // Note that this is a parameter, not constant, so can be set externally via
    // module projection (see below).
    param Fixed :=
        "    2  7 " &
        "    34   " &
        "358      " &
        "5 48     " &
        "   1   89" &
        "  2     6" &
        "24    7  " &
        " 9   52  " &
        "    671  ";

    // Map from the Fixed text value to the required moves.
    const ImposedIds :=
        Range(NumCells)
        ->{ cell: it, value: Fixed[it]->ToDigit() }
        ->TakeIf(0 <= value < N)
        ->Map(N * cell + value);
    const NumImposed := ImposedIds->Count();
    const ImposedFlags := Tensor.From(Range(NumMoves)->(it in ImposedIds));

    // Define a variable (tensor of bool) for the moves that have been made.
    var Flags from Tensor.Fill(false, NumMoves) def ImposedFlags;

    // Define a measure for the number of moves made.
    // Need to maximize this without violating constraints.
    msr NumMade := Flags.Values->Sum();

    // Define the constraints.

    // Require the imposed moves.
    con NeedImposed := ForEach(id:ImposedIds, Flags[id] = 1);

    // Require no "duplicates". That is, for each MovesByXxxYyy above, there
    // can be at most one move from each group.
    // NOTE: Here we use = 1. The other notebook uses <= 1 allowing it to"do the best
    // possible" when there is no complete solution. See below for examples.
    con OnePerRowCol := ForEach(c:MovesByRowCol, Sum(c, Flags[Id]) = 1);
    con OnePerValRow := ForEach(c:MovesByValRow, Sum(c, Flags[Id]) = 1);
    con OnePerValCol := ForEach(c:MovesByValCol, Sum(c, Flags[Id]) = 1);
    con OnePerValBlk := ForEach(c:MovesByValBlk, Sum(c, Flags[Id]) = 1);

    // The Board is for "pretty" display of the result.
    const Symbols := "_1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ";
    let Board :=
        Moves
        ->TakeIf(Flags[Id])
        ->SortUp(XRow, XCol)
        ->GroupBy(_:XRow)
        ->ForEach(With(row: it,
            ForEach(c: Range(N), With(i: First(row, XCol = c).ZVal + 1 ?? 0, Symbols[i:*1])))
            ->Text.Concat("|"))
        ->Text.Concat("\n");
};

Display the initial board with only the `Fixed` cells (required values) filled in.

In [3]:
Sudoku.Board;
Sudoku.NumMade;

_|_|_|_|2|_|_|7|_
_|_|_|_|3|4|_|_|_
3|5|8|_|_|_|_|_|_
5|_|4|8|_|_|_|_|_
_|_|_|1|_|_|_|8|9
_|_|2|_|_|_|_|_|6
2|4|_|_|_|_|7|_|_
_|9|_|_|_|5|2|_|_
_|_|_|_|6|7|1|_|_
24


Show the constraint values. If any are `false`, then the `Fixed` cells violate the constraints.

In [4]:
Sudoku.NeedImposed->All();
Sudoku.OnePerRowCol->All();
Sudoku.OnePerValRow->All();
Sudoku.OnePerValCol->All();
Sudoku.OnePerValBlk->All();
Sudoku.NumMade;

true
false
false
false
false
24


## Solve It

In [5]:
#!time
Sln := Sudoku->Maximize(NumMade, "highs");
Sln.Board;
Sln.NumMade;

Solver: HiGHS
4|6|1|5|2|8|9|7|3
7|2|9|6|3|4|8|5|1
3|5|8|7|1|9|6|4|2
5|1|4|8|9|6|3|2|7
6|7|3|1|5|2|4|8|9
9|8|2|4|7|3|5|1|6
2|4|6|9|8|1|7|3|5
1|9|7|3|4|5|2|6|8
8|3|5|2|6|7|1|9|4
81


Wall time: 145.7879ms

In [6]:
#!time
Sln2 := Sudoku->Maximize(NumMade, "glpk");
Sln2.Board;
Sln2.NumMade;

Solver: GLPK
4|6|1|5|2|8|9|7|3
7|2|9|6|3|4|8|5|1
3|5|8|7|1|9|6|4|2
5|1|4|8|9|6|3|2|7
6|7|3|1|5|2|4|8|9
9|8|2|4|7|3|5|1|6
2|4|6|9|8|1|7|3|5
1|9|7|3|4|5|2|6|8
8|3|5|2|6|7|1|9|4
81


Wall time: 86.0618ms

In [7]:
Sln.Board = Sln2.Board

true


## Partial Solve

This examples starts with the same required values, but with "2" added at the end of the 2nd row,
which is inconsistent with the solution. The result is the board cannot be completely filled in.
According to the solvers, the maximum number of cells that can be filled in is 79, leaving 2 unfilled.

Note that the two solvers find different configurations that achieve the maximum of 79 cells filled.

NOTE: the comments above are from the other notebook that use `<= ` in the constraints. This notebook
uses `= 1` (requiring the board to be completely filled), so each solver generates an error and
produces a `null` module as the result of `Maximize`.

In [8]:
Bad := Sudoku=>{
    Fixed:
        "    2  7 " &
        "    34  2" &
        "358      " &
        "5 48     " &
        "   1   89" &
        "  2     6" &
        "24    7  " &
        " 9   52  " &
        "    671  "
};

Bad.Board;
Bad.NumMade;

_|_|_|_|2|_|_|7|_
_|_|_|_|3|4|_|_|2
3|5|8|_|_|_|_|_|_
5|_|4|8|_|_|_|_|_
_|_|_|1|_|_|_|8|9
_|_|2|_|_|_|_|_|6
2|4|_|_|_|_|7|_|_
_|9|_|_|_|5|2|_|_
_|_|_|_|6|7|1|_|_
25


In [9]:
#!time
Sln := Bad->Maximize(NumMade, "highs");
Sln.Board;
Sln.NumMade;

Solver: HiGHS


Diagnostics:
  Error: Infeasible: contradictory constraints
  Error: Solving failed!



<null>
<null>


Wall time: 67.0234ms

In [10]:
#!time
Sln2 := Bad->Maximize(NumMade, "glpk");
Sln2.Board;
Sln2.NumMade;

Solver: GLPK


Diagnostics:
  Error: Solving failed!



<null>
<null>


Wall time: 71.3767ms

In [11]:
Sln.Board = Sln2.Board

true


## Hardest Sudoku

The "internet" claims this is the world's most difficult sudoku puzzle, taken from
[here](https://www.kristanix.com/sudokuepic/worlds-hardest-sudoku.php). The author of it,
Finnish mathematician Arto Inkala, calls it AI Escargot.

In [12]:
Hardest := Sudoku=>{
    Fixed:
        "1    7 9 " &
        " 3  2   8" &
        "  96  5  " &
        "  53  9  " &
        " 1  8   2" &
        "6    4   " &
        "3      1 " &
        " 4      7" &
        "  7   3  "
};

Hardest.NumImposed;

23


In [13]:
#!time
Sln := Hardest->Maximize(NumMade, "highs");
Sln.Board;
Sln.NumMade;

Solver: HiGHS
1|6|2|8|5|7|4|9|3
5|3|4|1|2|9|6|7|8
7|8|9|6|4|3|5|2|1
4|7|5|3|1|2|9|8|6
9|1|3|5|8|6|7|4|2
6|2|8|7|9|4|1|3|5
3|5|6|4|7|8|2|1|9
2|4|1|9|3|5|8|6|7
8|9|7|2|6|1|3|5|4
81


Wall time: 361.135ms

In [14]:
#!time
Sln2 := Hardest->Maximize(NumMade, "glpk");
Sln2.Board;
Sln2.NumMade;

Solver: GLPK
1|6|2|8|5|7|4|9|3
5|3|4|1|2|9|6|7|8
7|8|9|6|4|3|5|2|1
4|7|5|3|1|2|9|8|6
9|1|3|5|8|6|7|4|2
6|2|8|7|9|4|1|3|5
3|5|6|4|7|8|2|1|9
2|4|1|9|3|5|8|6|7
8|9|7|2|6|1|3|5|4
81


Wall time: 108.0457ms

In [15]:
Sln.Board = Sln2.Board

true


## Another difficult one

This is another difficult one, also authored by Arto Inkala.

In [16]:
Finnish := Sudoku=>{
    Fixed:
        "8        " &
        "  36     " &
        " 7  9 2  " &
        " 5   7   " &
        "    457  " &
        "   1   3 " &
        "  1    68" &
        "  85   1 " &
        " 9    4  "
};

In [17]:
#!time
Sln := Finnish->Maximize(NumMade, "highs");
Sln.Board;
Sln.NumMade;

Solver: HiGHS
8|1|2|7|5|3|6|4|9
9|4|3|6|8|2|1|7|5
6|7|5|4|9|1|2|8|3
1|5|4|2|3|7|8|9|6
3|6|9|8|4|5|7|2|1
2|8|7|1|6|9|5|3|4
5|2|1|9|7|4|3|6|8
4|3|8|5|2|6|9|1|7
7|9|6|3|1|8|4|5|2
81


Wall time: 97.6248ms

In [18]:
#!time
Sln2 := Finnish->Maximize(NumMade, "glpk");
Sln2.Board;
Sln2.NumMade;

Solver: GLPK
8|1|2|7|5|3|6|4|9
9|4|3|6|8|2|1|7|5
6|7|5|4|9|1|2|8|3
1|5|4|2|3|7|8|9|6
3|6|9|8|4|5|7|2|1
2|8|7|1|6|9|5|3|4
5|2|1|9|7|4|3|6|8
4|3|8|5|2|6|9|1|7
7|9|6|3|1|8|4|5|2
81


Wall time: 89.7634ms

In [19]:
Sln.Board = Sln2.Board

true


## Generate a Sudoku board from scratch

Here we specify only the first row. The solution isn't unique and indeed the two solvers produce different results. 

In [20]:
#!time
Sln := Sudoku=>{ Fixed: "123456789" }->Maximize(NumMade, "highs");
Sln.Board;
Sln.NumMade;

Solver: HiGHS
1|2|3|4|5|6|7|8|9
5|8|9|2|7|1|6|4|3
4|6|7|9|3|8|5|1|2
9|3|6|7|4|5|1|2|8
2|7|1|3|8|9|4|5|6
8|5|4|6|1|2|9|3|7
3|1|5|8|6|7|2|9|4
6|4|2|5|9|3|8|7|1
7|9|8|1|2|4|3|6|5
81


Wall time: 567.4761ms

In [21]:
#!time
Sln2 := Sudoku=>{ Fixed: "123456789" }->Maximize(NumMade, "glpk");
Sln2.Board;
Sln2.NumMade;

Solver: GLPK
1|2|3|4|5|6|7|8|9
4|5|9|7|2|8|6|3|1
8|7|6|3|9|1|2|4|5
2|1|4|8|6|7|9|5|3
3|9|5|1|4|2|8|6|7
7|6|8|9|3|5|1|2|4
6|3|2|5|1|9|4|7|8
9|4|7|2|8|3|5|1|6
5|8|1|6|7|4|3|9|2
81


Wall time: 152.6284ms

In [22]:
Sln.Board = Sln2.Board;

false


## Produce a rank 4 Sudoku board

Generate a board (specifying the first row) of rank 4.

NOTE: HiGHS does better with the `= 1` constraints but GLPK does significantly worse.
Compare the times here with the corresponding times in the other notebook.

In [23]:
#!time
Sln := Sudoku=>{ Fixed: "1234567890ABCDEF", M: 4 }->Maximize(NumMade, "highs");
Sln.Board;
Sln.NumMade;

Solver: HiGHS
1|2|3|4|5|6|7|8|9|0|A|B|C|D|E|F
F|E|8|0|D|C|1|B|3|7|2|6|5|A|9|4
D|C|7|B|F|3|A|9|E|8|5|4|0|2|6|1
A|9|6|5|2|E|4|0|F|D|1|C|7|B|8|3
7|F|D|A|1|B|5|E|C|9|0|8|6|4|3|2
E|3|C|9|A|8|F|D|7|6|4|2|B|0|1|5
2|B|0|6|C|9|3|4|D|5|F|1|E|8|A|7
8|4|5|1|6|2|0|7|B|E|3|A|F|C|D|9
6|D|F|E|B|A|C|5|0|4|9|7|3|1|2|8
C|A|B|8|E|7|D|F|1|2|6|3|9|5|4|0
0|7|9|3|8|4|2|1|A|F|E|5|D|6|B|C
5|1|4|2|9|0|6|3|8|C|B|D|A|F|7|E
4|6|E|7|0|F|B|C|2|A|8|9|1|3|5|D
B|0|A|F|7|1|8|2|5|3|D|E|4|9|C|6
9|8|1|D|3|5|E|6|4|B|C|F|2|7|0|A
3|5|2|C|4|D|9|A|6|1|7|0|8|E|F|B
256


Wall time: 10252.2966ms

In [24]:
#!time
Sln2 := Sudoku=>{ Fixed: "1234567890ABCDEF", M: 4 }->Maximize(NumMade, "glpk");
Sln2.Board;
Sln2.NumMade;

Solver: GLPK
1|2|3|4|5|6|7|8|9|0|A|B|C|D|E|F
C|9|7|0|2|A|4|F|D|5|6|E|8|3|B|1
B|6|5|8|3|9|D|E|F|C|1|2|4|A|7|0
F|E|D|A|B|1|C|0|8|7|4|3|9|2|6|5
3|8|1|D|4|C|9|7|6|2|E|A|5|0|F|B
5|A|F|B|1|E|6|2|3|9|0|C|D|8|4|7
6|C|0|2|A|5|8|D|7|F|B|4|3|9|1|E
4|7|9|E|F|0|3|B|5|1|8|D|2|6|C|A
A|5|6|F|9|4|0|1|C|B|D|8|E|7|2|3
0|4|8|9|7|2|B|C|E|3|5|F|6|1|A|D
2|1|C|3|D|8|E|A|0|6|7|9|B|F|5|4
D|B|E|7|6|F|5|3|4|A|2|1|0|C|9|8
7|F|4|C|E|D|2|5|A|8|3|6|1|B|0|9
E|0|B|1|C|3|F|4|2|D|9|7|A|5|8|6
9|D|2|5|8|B|A|6|1|4|F|0|7|E|3|C
8|3|A|6|0|7|1|9|B|E|C|5|F|4|D|2
256


Wall time: 11870.7776ms

In [25]:
Sln.Board = Sln2.Board;

false


## Produce a rank 5 Sudoku board

Now try rank 5, with Gurobi. HiGHS and GLPK don't do well on this one.
Gurobi even struggles with it. Note that this is severely under constrained.

In [26]:
#!time
Sln := Sudoku=>{ Fixed: "1234567890ABCDEFGHIJKLMNO", M: 5 }->Maximize(NumMade, "gurobi");
Sln.Board;
Sln.NumMade;

Solver: Gurobi
1|2|3|4|5|6|7|8|9|0|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O
K|C|7|B|M|N|4|A|G|L|H|J|F|0|5|6|O|9|D|1|3|8|2|E|I
6|H|0|I|N|E|5|O|D|M|9|K|G|L|1|3|4|C|8|2|B|A|7|F|J
G|A|D|J|O|I|3|F|K|B|2|7|8|4|N|E|5|0|L|M|9|H|6|C|1
L|8|F|E|9|C|J|2|H|1|O|I|3|M|6|N|B|K|A|7|D|G|5|4|0
J|O|M|2|3|D|0|7|5|N|4|6|L|F|K|H|C|B|E|G|1|I|8|A|9
N|6|4|7|F|M|2|B|E|K|0|D|O|H|G|8|I|A|1|9|L|J|C|5|3
9|5|K|A|H|J|O|G|I|3|1|E|N|C|8|M|L|F|6|D|4|B|0|2|7
E|I|1|G|D|9|A|L|C|8|3|5|M|B|J|2|0|O|7|4|6|K|F|H|N
0|L|B|C|8|H|6|1|F|4|I|A|2|9|7|J|N|5|3|K|G|M|E|O|D
4|E|G|K|J|F|H|9|N|7|D|3|A|5|I|0|8|L|M|C|O|2|1|B|6
C|9|H|8|I|K|G|M|O|E|L|1|0|2|F|B|6|N|J|5|A|D|3|7|4
O|1|5|M|A|2|B|3|4|D|G|N|6|8|C|9|E|7|K|I|J|F|H|0|L
3|D|2|L|6|0|8|C|J|5|7|O|9|K|B|1|H|4|F|A|E|N|G|I|M
F|7|N|0|B|A|L|I|1|6|J|M|4|E|H|O|D|G|2|3|C|9|K|8|5
I|4|A|5|K|8|M|N|0|C|E|H|J|G|D|L|1|2|9|F|7|3|O|6|B
D|N|J|3|C|7|I|K|2|G|6|F|B|A|L|5|M|E|0|O|8|1|4|9|H
2|B|6|O|E|5|9|4|L|J|M|0|1|7|3|D|K|8|H|N|F|C|I|G|A
M|0|8|F|7|3|1|H|B|A|5|4|K|O|9|C|J|I|G|6|N|E|D|L|2
H|G|9|1|L|O|E|D|6|F|8|C|I|N|2|A|7|3

Wall time: 29529.878ms